In [1]:
import os

In [2]:
%pwd

'e:\\wine_proj\\Wine_project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\wine_proj\\Wine_project'

In [5]:
# This Code is for Entity 
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml,create_directory

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_filpath = CONFIG_FILE_PATH,
                 params_filpath = PARAMS_FILE_PATH,
                 schema_filpath = SCHEMA_FILE_APTH):
        self.config = read_yaml(config_filpath)
        self.params = read_yaml(params_filpath)
        self.schema = read_yaml(schema_filpath)

        create_directory([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directory([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path= config.train_data_path,
            test_data_path = config.test_data_path,
            model_name=config.model_name,
            alpha= params.alpha,
            l1_ratio=params.l1_ratio,
            target_column= schema.name
        )

        return model_trainer_config
        

In [8]:


import pandas as pd
import os
from mlProject.logging import logger
from sklearn.linear_model import ElasticNet
import joblib

In [9]:
# This Code is for Components

class ModelTrainer:
    def __init__(self,config: ModelTrainerConfig):
        self.config = config

    

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column],axis=1)
        test_x = test_data.drop([self.config.target_column],axis=1)
        train_y = train_data([self.config.target_column])
        test_y = test_data([self.config.target_column])


        lr = ElasticNet(alpha=self.config.alpha,l1_ratio=self.config.l1_ratio,random_state=42)
        lr.fit(train_x,train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir,self.config.model_name))

In [10]:
# This code is for Pipeline only

try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

TypeError: get_model_trainer_config() missing 1 required positional argument: 'self'